In [1]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("data/Form.csv")


# Prepare data

In [3]:
# remove data not use
df = df.iloc[0:,[1,2,3]]

In [4]:
# we use age , gender and trasactions Store
df.head(5)

,Age,Gender,Store
0,20,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."
1,16,female,"BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA Co..."
2,22,male,"NAP's Coffee & Roasters, SongSarn, Yuanjai Cafe"
3,21,female,"NAP's Coffee & Roasters, Nap x Warin, Snoopcat..."
4,19,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."


In [5]:
# create functions to split transactions data store

def set_transactions(x):
    x = x.split(",")
    return x

In [6]:
# map values with age gender and store to format functions asscoations rule
df["Store"]  = df["Store"].apply(lambda x:set_transactions(x))


In [7]:
# map Age 
df['Age'] = df["Age"].map(lambda x:'10-20' if x <= 20 else '21-40' if x <= 40 else '41-60' if x <= 60 else '60+')

In [8]:
df

,Age,Gender,Store
0,10-20,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari..."
1,10-20,female,"[BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA..."
2,21-40,male,"[NAP's Coffee & Roasters, SongSarn, Yuanjai ..."
3,21-40,female,"[NAP's Coffee & Roasters, Nap x Warin, Snoop..."
4,10-20,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari..."
...,...,...,...
77,10-20,female,"[NAP's Coffee & Roasters, Nap x Warin]"
78,21-40,female,"[NAP's Coffee & Roasters, sangob, SongSarn, ..."
79,21-40,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and..."
80,41-60,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and..."


In [9]:
# map age gender to create Rule
age_groups = df.groupby('Age')
age_gender_rules = {}

In [26]:
# For Loop each group
for age_group, data in age_groups:
    for gender, gender_data in data.groupby('Gender'):
        # Use the TransactionEncoder to convert the data into a suitable format for the Apriori algorithm
        te = TransactionEncoder()
        te_ary = te.fit(gender_data['Store']).transform(gender_data['Store'])
        df_temp = pd.DataFrame(te_ary, columns=te.columns_)

        # Run the Apriori algorithm to find frequent item sets
        frequent_itemsets = apriori(df_temp, min_support=0.05, use_colnames=True)

        # Generate association rules from the frequent item sets
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
        # Store the association rules for each age group and gender in the dictionary
        age_gender_rules[(age_group, gender)] = rules

In [27]:
age_gender_rules.keys()

dict_keys([('10-20', 'female'), ('10-20', 'male'), ('21-40', 'female'), ('21-40', 'male'), ('41-60', 'female'), ('41-60', 'male')])

In [28]:
age_gender_rules

{('10-20',
  'female'):                       antecedents  \
 0     ( 11.11 Gallery and Coffee)   
 1       ( MiND-K coffee and bake)   
 2     ( 11.11 Gallery and Coffee)   
 3     ( 11.11 Gallery and Coffee)   
 4                   ( My Papilio)   
 ...                           ...   
 2879         (BalconyKiss Coffee)   
 2880             ( Snoopcat Cafe)   
 2881  ( 11.11 Gallery and Coffee)   
 2882                ( My Papilio)   
 2883    ( MiND-K coffee and bake)   
 
                                             consequents  antecedent support  \
 0                         ( LAVA  JAVA Coffee Roasters)                 0.1   
 1                           ( 11.11 Gallery and Coffee)                 0.2   
 2                             ( MiND-K coffee and bake)                 0.1   
 3                                         ( My Papilio)                 0.1   
 4                           ( 11.11 Gallery and Coffee)                 0.1   
 ...                                   

In [29]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,( Amarna),( Nap x Warin),1.0,0.5,0.5,0.5,1.0,0.00,1.0
1,( Nap x Warin),( Amarna),0.5,1.0,0.5,1.0,1.0,0.00,inf
2,( Phantae Coffee),( Amarna),0.5,1.0,0.5,1.0,1.0,0.00,inf
3,( Amarna),( Phantae Coffee),1.0,0.5,0.5,0.5,1.0,0.00,1.0
4,( Amarna),( REDCOFFEE),1.0,0.5,0.5,0.5,1.0,0.00,1.0
...,...,...,...,...,...,...,...,...,...
777,( Amarna),"( Saereesook, SongSarn, sangob, NAP's Coffee...",1.0,0.5,0.5,0.5,1.0,0.00,1.0
778,( SongSarn),"( Saereesook, Amarna, sangob, NAP's Coffee &...",0.5,0.5,0.5,1.0,2.0,0.25,inf
779,( sangob),"( Saereesook, Amarna, SongSarn, NAP's Coffee...",0.5,0.5,0.5,1.0,2.0,0.25,inf
780,(NAP's Coffee & Roasters),"( Saereesook, Amarna, SongSarn, sangob, Na...",0.5,0.5,0.5,1.0,2.0,0.25,inf
